In [1]:
import csv
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 20)
driver.get("https://kalimatimarket.gov.np/")

start_date = datetime.strptime('01/01/2022', '%m/%d/%Y')
end_date = datetime.strptime('12/31/2022', '%m/%d/%Y')

current_date = start_date
while current_date <= end_date:
    target_date = current_date.strftime('%m/%d/%Y')
    filename_date = target_date.replace('/', '_')
    filename = f'arrival_{filename_date}.csv'
    print(f"Processing {target_date}")

    try:
        # Find and set date input
        date_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='text'], input[type='date']")))
        date_input.clear()
        date_input.send_keys(target_date)

        # Click the "आगमन डाटा जाँच्नुहोस्" button
        arrival_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'मूल्यहरू जाँच गर्नुहोस्')]")))
        arrival_button.click()

        # Wait up to 20 seconds for table to update (avoid stale table)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table")))

        # Now, RE-FIND all rows fresh for each loop
        time.sleep(2)  # Give extra time for asynchronous JS data load

        table_data = []
        rows = driver.find_elements(By.CSS_SELECTOR, "table tr")  # <--- Fresh locate!
        for row in rows:
            cols = row.find_elements(By.TAG_NAME, "td")
            if cols:
                table_data.append([col.text.strip() for col in cols])

        # Check if there is any actual, meaningful data (not just empty rows)
        data_exists = any(any(cell for cell in row) for row in table_data)
        if data_exists:
            with open(filename, "w", newline='', encoding='utf-8') as f:
                writer = csv.writer(f)
                for row in table_data:
                    writer.writerow(row)
            print(f"Saved data to {filename}")
        else:
            print(f"No arrival data for {target_date} (table was blank or only headers)")

    except Exception as e:
        print(f"Error on {target_date}: {e}")

    current_date += timedelta(days=1)
    time.sleep(1)

driver.quit()
print("Done with sincere extraction for January 2025!")


Processing 01/01/2022
Saved data to arrival_01_01_2022.csv
Processing 01/02/2022
Saved data to arrival_01_02_2022.csv
Processing 01/03/2022
Saved data to arrival_01_03_2022.csv
Processing 01/04/2022
Saved data to arrival_01_04_2022.csv
Processing 01/05/2022
Saved data to arrival_01_05_2022.csv
Processing 01/06/2022
Saved data to arrival_01_06_2022.csv
Processing 01/07/2022
Saved data to arrival_01_07_2022.csv
Processing 01/08/2022
Saved data to arrival_01_08_2022.csv
Processing 01/09/2022
Saved data to arrival_01_09_2022.csv
Processing 01/10/2022
Saved data to arrival_01_10_2022.csv
Processing 01/11/2022
Saved data to arrival_01_11_2022.csv
Processing 01/12/2022
Saved data to arrival_01_12_2022.csv
Processing 01/13/2022
Saved data to arrival_01_13_2022.csv
Processing 01/14/2022
Saved data to arrival_01_14_2022.csv
Processing 01/15/2022
Saved data to arrival_01_15_2022.csv
Processing 01/16/2022
Saved data to arrival_01_16_2022.csv
Processing 01/17/2022
Saved data to arrival_01_17_2022.c

In [3]:
import pandas as pd
import os
from glob import glob

# ---------------------------------------------------------------
# STEP 1: Automatically detect current working directory
# ---------------------------------------------------------------
folder = os.getcwd()
print(f"📂 Working folder: {folder}")

# ---------------------------------------------------------------
# STEP 2: Find all CSV files starting with 'arrival_'
# ---------------------------------------------------------------
csv_files = sorted(glob(os.path.join(folder, "arrival_*.csv")))

print(f"✅ Found {len(csv_files)} CSV files to merge")

# ---------------------------------------------------------------
# STEP 3: Read, clean, and standardize each CSV
# ---------------------------------------------------------------
merged_data = []

for file in csv_files:
    try:
        # Extract date from filename, e.g. 'arrival_02_15_2022.csv' → '2022-02-15'
        basename = os.path.basename(file)
        date_str = basename.replace("arrival_", "").replace(".csv", "")
        # handle formats like arrival_1_2_2022.csv as well
        parts = date_str.split("_")
        if len(parts) == 3:
            month, day, year = parts
            formatted_date = f"{year}-{int(month):02d}-{int(day):02d}"
        else:
            formatted_date = None
        
        # Read CSV
        df = pd.read_csv(file)
        
        # Standardize columns (in case names vary or missing)
        df.columns = [col.strip().lower() for col in df.columns]
        
        # Select only first 6 columns if extra exist
        df = df.iloc[:, :6]
        
        # Assign correct column names
        df.columns = ['commodity', 'unit', 'minimum', 'maximum', 'average_price', 'extra'][:len(df.columns)]
        
        # Ensure required columns exist (fill missing)
        for col in ['commodity', 'unit', 'minimum', 'maximum', 'average_price']:
            if col not in df.columns:
                df[col] = None
        
        # Add Date column
        df['Date'] = formatted_date
        
        # Reorder columns
        df = df[['Date', 'commodity', 'unit', 'minimum', 'maximum', 'average_price']]
        
        # Append to list
        merged_data.append(df)

    except Exception as e:
        print(f"⚠️ Error reading {file}: {e}")

# ---------------------------------------------------------------
# STEP 4: Merge all DataFrames
# ---------------------------------------------------------------
final_df = pd.concat(merged_data, ignore_index=True)
final_df = final_df.drop_duplicates().reset_index(drop=True)

# ---------------------------------------------------------------
# STEP 5: Save merged CSV
# ---------------------------------------------------------------
output_file = os.path.join(folder, "arrival_2022_full.csv")
final_df.to_csv(output_file, index=False, encoding='utf-8-sig')

print("✅ Merging completed successfully!")
print(f"💾 File saved as: {output_file}")
print("Final shape:", final_df.shape)
print(final_df.head())


📂 Working folder: D:\Data Science\First Semester\Machine Learning\Group Project\Kalimati
✅ Found 367 CSV files to merge
✅ Merging completed successfully!
💾 File saved as: D:\Data Science\First Semester\Machine Learning\Group Project\Kalimati\arrival_2022_full.csv
Final shape: (36212, 6)
         Date             commodity    unit    minimum    maximum  \
0  2022-01-01  गोलभेडा ठूलो(भारतीय)    केजी  रू १०१.००  रू १०१.००   
1  2022-01-01    गोलभेडा सानो(लोकल)  के.जी.   रू ४०.००   रू ५०.००   
2  2022-01-01    गोलभेडा सानो(टनेल)   के जी   रू ५०.००   रू ६०.००   
3  2022-01-01  गोलभेडा सानो(भारतीय)   के जी   रू ५५.००   रू ६५.००   
4  2022-01-01    गोलभेडा सानो(तराई)   के जी   रू ६०.००   रू ७०.००   

  average_price  
0     रू १०१.००  
1      रू ४५.००  
2      रू ५५.००  
3      रू ६०.००  
4      रू ६५.००  


C:\Users\User\AppData\Local\Temp\ipykernel_20552\257274747.py:37: DtypeWarning: Columns (2,3,4,8,10,15,18,20,21,24,26,27,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


In [4]:
import pandas as pd
import os
from glob import glob

# ---------------------------------------------------------------
# STEP 1: Detect current working directory
# ---------------------------------------------------------------
folder = os.getcwd()
print(f"📂 Current working directory: {folder}")

# ---------------------------------------------------------------
# STEP 2: Get all arrival CSV files
# ---------------------------------------------------------------
csv_files = sorted(glob(os.path.join(folder, "arrival_*.csv")))
print(f"✅ Found {len(csv_files)} CSV files to merge")

# ---------------------------------------------------------------
# STEP 3: Define correct column names
# ---------------------------------------------------------------
columns = ['commodity', 'unit', 'minimum', 'maximum', 'average_price']

merged_data = []

# ---------------------------------------------------------------
# STEP 4: Read each file (no header) and add date from filename
# ---------------------------------------------------------------
for file in csv_files:
    try:
        # Extract date from filename, e.g. "arrival_03_15_2022.csv" → "2022-03-15"
        basename = os.path.basename(file)
        date_str = basename.replace("arrival_", "").replace(".csv", "")
        parts = date_str.split("_")
        if len(parts) == 3:
            month, day, year = parts
            formatted_date = f"{year}-{int(month):02d}-{int(day):02d}"
        else:
            formatted_date = None

        # Read CSV with no header
        df = pd.read_csv(file, header=None)

        # Assign column names manually
        df.columns = columns[:len(df.columns)]

        # Add Date column
        df['Date'] = formatted_date

        # Reorder columns (Date first)
        df = df[['Date'] + columns]

        merged_data.append(df)

    except Exception as e:
        print(f"⚠️ Error reading {file}: {e}")

# ---------------------------------------------------------------
# STEP 5: Merge all data into one DataFrame
# ---------------------------------------------------------------
final_df = pd.concat(merged_data, ignore_index=True)

# ---------------------------------------------------------------
# STEP 6: Clean & Save
# ---------------------------------------------------------------
final_df = final_df.drop_duplicates().reset_index(drop=True)

# Convert numeric columns if needed
for col in ['minimum', 'maximum', 'average_price']:
    final_df[col] = pd.to_numeric(final_df[col], errors='coerce')

# Save as single CSV
output_file = os.path.join(folder, "arrival_2022_full.csv")
final_df.to_csv(output_file, index=False, encoding='utf-8-sig')

print("✅ Merge complete!")
print(f"💾 Saved as: {output_file}")
print("Final shape:", final_df.shape)
print(final_df.head())


📂 Current working directory: D:\Data Science\First Semester\Machine Learning\Group Project\Kalimati
✅ Found 366 CSV files to merge
⚠️ Error reading D:\Data Science\First Semester\Machine Learning\Group Project\Kalimati\arrival_02_26_2022.csv: "['unit', 'minimum', 'maximum', 'average_price'] not in index"
⚠️ Error reading D:\Data Science\First Semester\Machine Learning\Group Project\Kalimati\arrival_03_04_2022.csv: "['unit', 'minimum', 'maximum', 'average_price'] not in index"
⚠️ Error reading D:\Data Science\First Semester\Machine Learning\Group Project\Kalimati\arrival_05_13_2022.csv: "['unit', 'minimum', 'maximum', 'average_price'] not in index"
⚠️ Error reading D:\Data Science\First Semester\Machine Learning\Group Project\Kalimati\arrival_11_20_2022.csv: "['unit', 'minimum', 'maximum', 'average_price'] not in index"
⚠️ Error reading D:\Data Science\First Semester\Machine Learning\Group Project\Kalimati\arrival_merge.csv: "['average_price'] not in index"
✅ Merge complete!
💾 Saved as:

In [5]:
import pandas as pd
import os
from glob import glob

# ---------------------------------------------------------------
# STEP 1: Working directory (your notebook folder)
# ---------------------------------------------------------------
folder = os.getcwd()
print(f"📂 Current working directory: {folder}")

# ---------------------------------------------------------------
# STEP 2: Get all CSV files (skip already merged ones)
# ---------------------------------------------------------------
csv_files = sorted(glob(os.path.join(folder, "arrival_*.csv")))
csv_files = [f for f in csv_files if "merge" not in f and "full" not in f]

print(f"✅ Found {len(csv_files)} CSV files to merge")

# ---------------------------------------------------------------
# STEP 3: Define standard column names
# ---------------------------------------------------------------
standard_cols = ['commodity', 'unit', 'minimum', 'maximum', 'average_price']

merged_data = []

# ---------------------------------------------------------------
# STEP 4: Loop through all files
# ---------------------------------------------------------------
for file in csv_files:
    try:
        # Extract date from filename
        basename = os.path.basename(file)
        date_str = basename.replace("arrival_", "").replace(".csv", "")
        parts = date_str.split("_")
        if len(parts) == 3:
            month, day, year = parts
            formatted_date = f"{year}-{int(month):02d}-{int(day):02d}"
        else:
            formatted_date = None

        # Try reading with flexible delimiters
        try:
            df = pd.read_csv(file, header=None, engine='python')
        except:
            df = pd.read_csv(file, header=None, delimiter=';', engine='python')
        
        # Drop completely empty columns
        df = df.dropna(axis=1, how='all')
        
        # Keep only up to 5 columns (extra columns are garbage)
        df = df.iloc[:, :5]

        # Assign standard column names
        df.columns = standard_cols[:len(df.columns)]

        # Fill missing columns if file had fewer columns
        for col in standard_cols:
            if col not in df.columns:
                df[col] = None

        # Add date column
        df['Date'] = formatted_date

        # Reorder
        df = df[['Date'] + standard_cols]

        # Clean up whitespace in text columns
        for col in ['commodity', 'unit']:
            df[col] = df[col].astype(str).str.strip()

        merged_data.append(df)

    except Exception as e:
        print(f"⚠️ Error reading {file}: {e}")

# ---------------------------------------------------------------
# STEP 5: Combine all data
# ---------------------------------------------------------------
final_df = pd.concat(merged_data, ignore_index=True)
final_df = final_df.drop_duplicates().reset_index(drop=True)

# Convert numeric columns to proper dtype
for col in ['minimum', 'maximum', 'average_price']:
    final_df[col] = pd.to_numeric(final_df[col], errors='coerce')

# ---------------------------------------------------------------
# STEP 6: Save final merged CSV
# ---------------------------------------------------------------
output_file = os.path.join(folder, "arrival_2022_full.csv")
final_df.to_csv(output_file, index=False, encoding='utf-8-sig')

print("✅ Merge complete!")
print(f"💾 Saved as: {output_file}")
print("Final shape:", final_df.shape)
print(final_df.head(10))


📂 Current working directory: D:\Data Science\First Semester\Machine Learning\Group Project\Kalimati
✅ Found 365 CSV files to merge
✅ Merge complete!
💾 Saved as: D:\Data Science\First Semester\Machine Learning\Group Project\Kalimati\arrival_2022_full.csv
Final shape: (35088, 6)
         Date              commodity    unit  minimum  maximum  average_price
0  2022-01-01   गोलभेडा ठूलो(नेपाली)  के.जी.      NaN      NaN            NaN
1  2022-01-01   गोलभेडा ठूलो(भारतीय)    केजी      NaN      NaN            NaN
2  2022-01-01     गोलभेडा सानो(लोकल)  के.जी.      NaN      NaN            NaN
3  2022-01-01     गोलभेडा सानो(टनेल)   के जी      NaN      NaN            NaN
4  2022-01-01   गोलभेडा सानो(भारतीय)   के जी      NaN      NaN            NaN
5  2022-01-01     गोलभेडा सानो(तराई)   के जी      NaN      NaN            NaN
6  2022-01-01               आलु रातो  के.जी.      NaN      NaN            NaN
7  2022-01-01       आलु रातो(भारतीय)   के जी      NaN      NaN            NaN
8  2022-01-01       

In [6]:
import pandas as pd
import os
from glob import glob

# ---------------------------------------------------------------
# STEP 1: Working directory (your notebook folder)
# ---------------------------------------------------------------
folder = os.getcwd()
print(f"📂 Current working directory: {folder}")

# ---------------------------------------------------------------
# STEP 2: Get all CSV files (skip already merged ones)
# ---------------------------------------------------------------
csv_files = sorted(glob(os.path.join(folder, "arrival_*.csv")))
csv_files = [f for f in csv_files if "merge" not in f and "full" not in f]

print(f"✅ Found {len(csv_files)} CSV files to merge")

# ---------------------------------------------------------------
# STEP 3: Define standard column names
# ---------------------------------------------------------------
standard_cols = ['commodity', 'unit', 'minimum', 'maximum', 'average_price']

merged_data = []

# ---------------------------------------------------------------
# STEP 4: Loop through all files
# ---------------------------------------------------------------
for file in csv_files:
    try:
        # Extract date from filename
        basename = os.path.basename(file)
        date_str = basename.replace("arrival_", "").replace(".csv", "")
        parts = date_str.split("_")
        if len(parts) == 3:
            month, day, year = parts
            formatted_date = f"{year}-{int(month):02d}-{int(day):02d}"
        else:
            formatted_date = None

        # Try reading with flexible delimiters
        try:
            df = pd.read_csv(file, header=None, engine='python')
        except:
            df = pd.read_csv(file, header=None, delimiter=';', engine='python')
        
        # Drop completely empty columns
        df = df.dropna(axis=1, how='all')
        
        # Keep only up to 5 columns (ignore extra garbage)
        df = df.iloc[:, :5]

        # Assign standard column names
        df.columns = standard_cols[:len(df.columns)]

        # Fill missing columns if file had fewer columns
        for col in standard_cols:
            if col not in df.columns:
                df[col] = None

        # Add Date column
        df['Date'] = formatted_date

        # Reorder columns
        df = df[['Date'] + standard_cols]

        # Clean text columns
        for col in ['commodity', 'unit', 'minimum', 'maximum', 'average_price']:
            df[col] = df[col].astype(str).str.strip()

        merged_data.append(df)

    except Exception as e:
        print(f"⚠️ Error reading {file}: {e}")

# ---------------------------------------------------------------
# STEP 5: Combine all dataframes
# ---------------------------------------------------------------
final_df = pd.concat(merged_data, ignore_index=True)
final_df = final_df.drop_duplicates().reset_index(drop=True)

# ---------------------------------------------------------------
# STEP 6: Save merged CSV (keep as text)
# ---------------------------------------------------------------
output_file = os.path.join(folder, "arrival_2022_full.csv")
final_df.to_csv(output_file, index=False, encoding='utf-8-sig')

print("✅ Merge complete!")
print(f"💾 Saved as: {output_file}")
print("Final shape:", final_df.shape)
print(final_df.head(10))


📂 Current working directory: D:\Data Science\First Semester\Machine Learning\Group Project\Kalimati
✅ Found 365 CSV files to merge
✅ Merge complete!
💾 Saved as: D:\Data Science\First Semester\Machine Learning\Group Project\Kalimati\arrival_2022_full.csv
Final shape: (35088, 6)
         Date              commodity    unit    minimum    maximum  \
0  2022-01-01   गोलभेडा ठूलो(नेपाली)  के.जी.  रू ११०.००  रू १२०.००   
1  2022-01-01   गोलभेडा ठूलो(भारतीय)    केजी  रू १०१.००  रू १०१.००   
2  2022-01-01     गोलभेडा सानो(लोकल)  के.जी.   रू ४०.००   रू ५०.००   
3  2022-01-01     गोलभेडा सानो(टनेल)   के जी   रू ५०.००   रू ६०.००   
4  2022-01-01   गोलभेडा सानो(भारतीय)   के जी   रू ५५.००   रू ६५.००   
5  2022-01-01     गोलभेडा सानो(तराई)   के जी   रू ६०.००   रू ७०.००   
6  2022-01-01               आलु रातो  के.जी.   रू ३२.००   रू ३६.००   
7  2022-01-01       आलु रातो(भारतीय)   के जी   रू २५.००   रू ३०.००   
8  2022-01-01               आलु सेतो  के.जी.   रू २२.००   रू २५.००   
9  2022-01-01  प्याज स

In [8]:
df = pd.read_csv('arrival_2022_full.csv')
df.head()

,Date,commodity,unit,minimum,maximum,average_price
0,2022-01-01,गोलभेडा ठूलो(नेपाली),के.जी.,रू ११०.००,रू १२०.००,रू ११५.००
1,2022-01-01,गोलभेडा ठूलो(भारतीय),केजी,रू १०१.००,रू १०१.००,रू १०१.००
2,2022-01-01,गोलभेडा सानो(लोकल),के.जी.,रू ४०.००,रू ५०.००,रू ४५.००
3,2022-01-01,गोलभेडा सानो(टनेल),के जी,रू ५०.००,रू ६०.००,रू ५५.००
4,2022-01-01,गोलभेडा सानो(भारतीय),के जी,रू ५५.००,रू ६५.००,रू ६०.००


In [9]:
df.tail()

,Date,commodity,unit,minimum,maximum,average_price
35083,2022-12-31,ताजा माछा(रहु),के जी,रू ३२०.००,रू ३४०.००,रू ३३०.००
35084,2022-12-31,ताजा माछा(बचुवा),के जी,रू २४०.००,रू २५०.००,रू २४५.००
35085,2022-12-31,ताजा माछा(छडी),के जी,रू २५०.००,रू २६०.००,रू २५५.००
35086,2022-12-31,ताजा माछा(मुंगरी),के जी,रू २६०.००,रू २८०.००,रू २७०.००
35087,2022-12-31,रुख टमाटर,केजी,रू ७०.००,रू ८०.००,रू ७५.००


In [10]:
df_tomato_big = df[df['commodity']=='गोलभेडा ठूलो(नेपाली)']

In [11]:
df_tomato_big

,Date,commodity,unit,minimum,maximum,average_price
0,2022-01-01,गोलभेडा ठूलो(नेपाली),के.जी.,रू ११०.००,रू १२०.००,रू ११५.००
109,2022-01-02,गोलभेडा ठूलो(नेपाली),के.जी.,रू ११०.००,रू १२०.००,रू ११५.००
216,2022-01-03,गोलभेडा ठूलो(नेपाली),के.जी.,रू ९०.००,रू १००.००,रू ९५.००
323,2022-01-04,गोलभेडा ठूलो(नेपाली),के.जी.,रू ९०.००,रू १००.००,रू ९६.६७
430,2022-01-05,गोलभेडा ठूलो(नेपाली),के.जी.,रू ८०.००,रू ९०.००,रू ८६.६७
...,...,...,...,...,...,...
34521,2022-12-27,गोलभेडा ठूलो(नेपाली),के.जी.,रू ४५.००,रू ५५.००,रू ५०.००
34633,2022-12-28,गोलभेडा ठूलो(नेपाली),के.जी.,रू ५०.००,रू ६०.००,रू ५५.००
34747,2022-12-29,गोलभेडा ठूलो(नेपाली),के.जी.,रू ४०.००,रू ५०.००,रू ४५.००
34860,2022-12-30,गोलभेडा ठूलो(नेपाली),के.जी.,रू ४०.००,रू ५०.००,रू ४५.००


In [12]:

# Mapping Nepali to English digits
nep_to_eng = str.maketrans('०१२३४५६७८९', '0123456789')

# Apply translation to all string columns
# df1 = df1.applymap(lambda x: x.translate(nep_to_eng) if isinstance(x, str) else x)
df_tomato_big = df_tomato_big.apply(lambda col: col.map(lambda x: x.translate(nep_to_eng) if isinstance(x, str) else x))



In [13]:
df_tomato_big.head()

,Date,commodity,unit,minimum,maximum,average_price
0,2022-01-01,गोलभेडा ठूलो(नेपाली),के.जी.,रू 110.00,रू 120.00,रू 115.00
109,2022-01-02,गोलभेडा ठूलो(नेपाली),के.जी.,रू 110.00,रू 120.00,रू 115.00
216,2022-01-03,गोलभेडा ठूलो(नेपाली),के.जी.,रू 90.00,रू 100.00,रू 95.00
323,2022-01-04,गोलभेडा ठूलो(नेपाली),के.जी.,रू 90.00,रू 100.00,रू 96.67
430,2022-01-05,गोलभेडा ठूलो(नेपाली),के.जी.,रू 80.00,रू 90.00,रू 86.67


In [14]:
df_tomato_big['minimum']= df_tomato_big['minimum'].str.split(' ').str[1]
df_tomato_big['maximum']= df_tomato_big['maximum'].str.split(' ').str[1]
df_tomato_big['average_price']= df_tomato_big['average_price'].str.split(' ').str[1]

In [15]:
df_tomato_big

,Date,commodity,unit,minimum,maximum,average_price
0,2022-01-01,गोलभेडा ठूलो(नेपाली),के.जी.,110.00,120.00,115.00
109,2022-01-02,गोलभेडा ठूलो(नेपाली),के.जी.,110.00,120.00,115.00
216,2022-01-03,गोलभेडा ठूलो(नेपाली),के.जी.,90.00,100.00,95.00
323,2022-01-04,गोलभेडा ठूलो(नेपाली),के.जी.,90.00,100.00,96.67
430,2022-01-05,गोलभेडा ठूलो(नेपाली),के.जी.,80.00,90.00,86.67
...,...,...,...,...,...,...
34521,2022-12-27,गोलभेडा ठूलो(नेपाली),के.जी.,45.00,55.00,50.00
34633,2022-12-28,गोलभेडा ठूलो(नेपाली),के.जी.,50.00,60.00,55.00
34747,2022-12-29,गोलभेडा ठूलो(नेपाली),के.जी.,40.00,50.00,45.00
34860,2022-12-30,गोलभेडा ठूलो(नेपाली),के.जी.,40.00,50.00,45.00
